In [13]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [31]:
current_total_atom_supply = 300000000
current_community_pool_balance = 1140000
current_security_subsidy_rate = 13.5
current_security_subsidy = (current_total_atom_supply * (current_security_subsidy_rate/100))/12
security_subsidy_decay = 10
security_subsidy_termination = 36 
issuance = 10000000
issuance_reduction_epoch_rate = 1
issuance_reduction_amount = 12
steady_state_issuance = 300000
time_until_steady_state_issuance = 36
distrbution_module_pct_to_community_pool = 5
timeline_length = 38

In [32]:
print(current_total_atom_supply, current_security_subsidy)

300000000 3375000.0


### Form the logic for each Category

In [ ]:
# Outflow - Security Subsidy
for month in range(timeline_length + 1):
    if month > security_subsidy_termination : 
        print(0)
    else: 
        print(current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(1-distrbution_module_pct_to_community_pool/100))

In [60]:
def calaculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool):
    if month > security_subsidy_termination : 
        return 0
    else: 
        return(current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(1-distrbution_module_pct_to_community_pool/100))

outflow_security_subsidy = calaculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)


In [ ]:
# Outflow - Distribution to Community Pool
for month in range(timeline_length + 1):
    if month > security_subsidy_termination : 
        print(0)
    else: 
        print(current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(distrbution_module_pct_to_community_pool/100))


In [65]:
def calaculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool):
    if month > security_subsidy_termination : 
        return 0
    else: 
        return(current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(distrbution_module_pct_to_community_pool/100))
    
outflow_distribution_community_pool = calaculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)


In [ ]:
# Inflow - Issued ATOM (new)
for month in range(timeline_length + 1):
    if month >= time_until_steady_state_issuance : 
        print(steady_state_issuance)
    else: 
        print((issuance-200000) * pow((1-issuance_reduction_amount/100),month)+200000)

In [75]:
def calaculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance):
    if month >= time_until_steady_state_issuance : 
        return steady_state_issuance
    else: 
        return((issuance-200000) * pow((1-issuance_reduction_amount/100),month)+200000)
    
inflow_issued_atom_new = calaculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance)


In [78]:
# Total	- Treasury Pool Balance
    
def calaculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new):
    total_outflow = outflow_distribution_community_pool + outflow_security_subsidy
    total_treasury_pool_balance = inflow_issued_atom_new - total_outflow
    return total_treasury_pool_balance

total_treasury_pool_balance = calaculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new)

In [ ]:
# Total	Community Pool Balance
current_total_comm_pool = current_community_pool_balance
for month in range(timeline_length + 1):
    if month > security_subsidy_termination : 
        distributed_comm_pool = 0
    else: 
        distributed_comm_pool = current_security_subsidy * pow((1-security_subsidy_decay/100),month)*(distrbution_module_pct_to_community_pool/100)
    
    current_total_comm_pool += distributed_comm_pool
    print(current_total_comm_pool)

In [ ]:
# Total	- Cumulatively Issued ATOM (new)
total_issued = 0

for month in range(timeline_length + 1):
    if month >= time_until_steady_state_issuance : 
        issued = (steady_state_issuance)
    else: 
        issued = ((issuance-200000) * pow((1-issuance_reduction_amount/100),month)+200000)

    total_issued +=  issued
    print(total_issued)

In [ ]:
# Total - Total ATOM Supply
total_atom_supply = current_total_atom_supply

for month in range(timeline_length + 1):
    if month >= time_until_steady_state_issuance : 
        issued_supply = (steady_state_issuance)
    else: 
        issued_supply = (issuance-200000) * pow((1-issuance_reduction_amount/100),month)+200000

    total_atom_supply += issued_supply
    print('\nMonth ', month, ' : ', total_atom_supply)

In [ ]:
# Old_Regime 
total_atom_supply_old = current_total_atom_supply
issued_atom_old = 0
cumulatively_issued_atom_old = issued_atom_old

for month in range(timeline_length + 1):
    total_atom_supply_old += issued_atom_old
    issued_atom_old = total_atom_supply_old*current_security_subsidy_rate/100/12
    cumulatively_issued_atom_old += issued_atom_old


    print('\nMonth : ', month, '\ntotal_atom_supply_old : ', total_atom_supply_old, '\n', 'issued_atom_old : ', issued_atom_old, '\n', 'cumulatively_issued_atom_old : ', cumulatively_issued_atom_old )

### Combine the logics into one function

In [82]:
total_treasury_pool_balance = 0
current_total_comm_pool = current_community_pool_balance

for month in range(timeline_length + 1):
    outflow_security_subsidy = calaculate_outflow_security_subsidy(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)
    outflow_distribution_community_pool = calaculate_outflow_distribution_community_pool(month, security_subsidy_termination, current_security_subsidy, security_subsidy_decay, distrbution_module_pct_to_community_pool)
    inflow_issued_atom_new = calaculate_inflow_issued_atom_new(month, time_until_steady_state_issuance, issuance, issuance_reduction_amount, steady_state_issuance)
    total_treasury_pool_balance += calaculate_total_treasury_pool_balance(outflow_distribution_community_pool, outflow_security_subsidy, inflow_issued_atom_new)
    current_total_comm_pool += outflow_distribution_community_pool
    



    print(month, outflow_security_subsidy, outflow_distribution_community_pool, inflow_issued_atom_new, total_treasury_pool_balance, current_total_comm_pool)

0 3206250.0 168750.0 10000000.0 6625000.0 1308750.0
1 2885625.0 151875.0 8824000.0 12411500.0 1460625.0
2 2597062.5 136687.5 7789120.0 17466870.0 1597312.5
3 2337356.2500000005 123018.75000000003 6878425.6 21884920.6 1720331.25
4 2103620.625 110716.875 6077014.528 25747597.628000002 1831048.125
5 1893258.5625000002 99645.18750000001 5371772.78464 29126466.66264 1930693.3125
6 1703932.70625 89680.66875000001 4751160.050483201 32084013.338123202 2020373.98125
7 1533539.4356250002 80712.60187500002 4205020.844425216 34674782.14504842 2101086.583125
8 1380185.4920625002 72641.34168750001 3724418.34309419 36946373.65439261 2173727.9248125
9 1242166.94285625 65377.207518750016 3301488.1419228874 38940317.6459405 2239105.13233125
10 1117950.2485706252 58839.486766875016 2929309.564892141 40692837.47549514 2297944.6190981246
11 1006155.2237135626 52955.53809018751 2601792.4171050843 42235519.13079647 2350900.157188312
12 905539.7013422066 47659.98428116877 2313577.327052474 43595896.772225566 

### Populate the result into a table

In [26]:
category_dict = {}
category_dict['Outflow'] = ["Security Subsidy", "Distribution to Community Pool"] 
category_dict['Inflow'] = ['Issued ATOM (new)']
category_dict['Total'] = ['Treasury Pool Balance', 'Community Pool Balance', 'Cumulatively Issued ATOM (new)', 'Total ATOM Supply']
category_dict['Old_Regime'] = ['Total ATOM Supply (old)', 'Issued ATOM (old)', 'Cumulatively Issued ATOM (old)']

pp.pprint(category_dict) 

{   'Inflow': ['Issued ATOM (new)'],
    'Old_Regime': [   'Total ATOM Supply (old)',
                      'Issued ATOM (old)',
                      'Cumulatively Issued ATOM (old)'],
    'Outflow': ['Security Subsidy', 'Distribution to Community Pool'],
    'Total': [   'Treasury Pool Balance',
                 'Community Pool Balance',
                 'Cumulatively Issued ATOM (new)',
                 'Total ATOM Supply']}


In [20]:
table = [['Category', 'Month', '0']]

In [22]:
# Populate the table based on the category_dict and timeline_length
for category, subcategories in category_dict.items():
    for subcategory in subcategories:
        for i in range(timeline_length + 1):
            table.append([category, subcategory, i])

In [ ]:
   <!-- <div id="chart"></div> -->

    <!-- Include Plotly.js -->
    <!-- <script src="https://cdn.plot.ly/plotly-latest.min.js"></script> -->

    <!-- Render the chart using the JSON data -->
    <!-- <script>
        var chartData = {{ chart_data | tojson | safe }};
        Plotly.newPlot('chart', JSON.parse(chartData), {});
    </script> -->